In [13]:
# 【1-1】必要なライブラリのインポート
from openai import AzureOpenAI  # Azure OpenAI Service を扱うためのクライアント
import pandas as pd  # CSVやテーブル形式のデータを扱うためのライブラリ
import numpy as np   # 数値演算ライブラリ（ベクトルや行列演算に使用）
from dotenv import load_dotenv
import os
from tqdm import tqdm 
import ast  # 文字列をPythonリスト等に評価するのに使う

In [2]:
# Load environment variables from .env file
load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
API_VERSION = os.getenv("API_VERSION")
DEPLOYMENT_ID_FOR_CHAT_COMPLETION = os.getenv("DEPLOYMENT_ID_FOR_CHAT_COMPLETION")
DEPLOYMENT_ID_FOR_EMBEDDING = os.getenv("DEPLOYMENT_ID_FOR_EMBEDDING")

# 【1-3】AzureOpenAI のクライアントを初期化
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=API_VERSION
)


In [3]:
def cosine_similarity(vec_a, vec_b):
    """
    コサイン類似度を計算する関数。
    例: cosine_similarity([1,0,0], [0.7,0.7,0])
    ベクトルaとベクトルbの内積を、両ベクトルのノルムの積で割ることで求める。
    """
    a = np.array(vec_a)  # listをnp.arrayに変換
    b = np.array(vec_b)
    # dot()で内積を計算し、linalg.norm()でベクトルの長さ(ノルム)を計算
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [4]:
def str_to_list(str_embedding):
    return ast.literal_eval(str_embedding)


In [5]:
#set_path
pdf_folder_path  = "pdf/"
text_folder_path = "text/"
csv_folder_path  = "csv/"

pdf_text_folder_path = text_folder_path + "pdf_text/"
pdf_summarize_text_folder_path = text_folder_path + "pdf_summarize_text/"

In [6]:
#read_vector
vector_df = pd.read_csv(csv_folder_path + "vector.csv")
vector_df["embedding"] = vector_df["embedding"].apply(str_to_list)
vector_df.head(2)  # 上位5行だけ表示

,Unnamed: 0,file_path,embedding
0,0,text/pdf_summarize_text/1.txt,"[0.023383982479572296, -0.01869707927107811, -..."
1,1,text/pdf_summarize_text/10.txt,"[-0.0005391213926486671, 0.009955545887351036,..."


In [10]:
#read_query
query_df = pd.read_csv(csv_folder_path + "query.csv",index_col=0)
query_df["answer"] =""
query_df.head(2)  # 上位5行だけ表示



,problem,answer
index,,
0,高松コンストラクショングループの2025年3月期の受注高の計画は前期比何倍か、小数第三位を四...,
1,株式会社キッツの取締役の報酬のうち株式報酬の割合は何％？,


In [19]:
for idx,row in  tqdm(query_df.iterrows()):

    user_query = row.problem

    # --- ユーザークエリをEmbedding ---
    response = client.embeddings.create(
        input=user_query,
        model=DEPLOYMENT_ID_FOR_EMBEDDING
    )
    query_vector = response.data[0].embedding


    # --- ユーザークエリと類似度の高い文書を取得 ---
    selected_index = vector_df["embedding"].apply(lambda doc_vec: cosine_similarity(doc_vec, query_vector))\
                                    .sort_values(ascending=False).head(3).index
    selected_records = vector_df.loc[selected_index]


    # --- プロンプトの作成 ---
    doc_context = ""  
    for idx, row in selected_records.iterrows():
        # 実際にファイルを読み込む
        with open(row['file_path'], "r", encoding="utf-8") as f:
            content = f.read()
        # ファイルパスと本文をまとめてコンテキストに追加
        doc_context += f"\n=== ソース: {row['file_path']} ===\n{content}\n"

        system_content = (
            "あなたは優秀なAIアシスタントです。"
            "ユーザーが与えた情報だけをもとに回答してください。"
            "情報がコンテキストに含まれない場合は『わかりません』と答えてください。"
            "回答は最大54トークンにしてください。"
        )
        user_content = (
            "以下のコンテキストを参考に回答をしてください。\n"  # 命令文
            "質問:\n"  # 質問文の出力
            f"{user_query}\n\n"
            "コンテキスト:\n"
            f"{doc_context}"
        )
        messages = [
            {"role": "system", "content": system_content},  # システムメッセージ
            {"role": "user", "content": user_content},       # ユーザーメッセージ
        ]

        answer = client.chat.completions.create(
            model=DEPLOYMENT_ID_FOR_CHAT_COMPLETION,
            messages=messages,
            max_tokens=4000,     # 応答に使えるトークン数上限
            temperature=0.7,     # 創造性（0~1）
            top_p=0.95,          # nucleus sampling
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False
        ).choices[0].message.content  # 応答本文

        #save
        query_df.loc[idx,"answer"] = answer

        # if not "わかりません" in answer:

        #     print("\n===== チャットのコンテキスト =====\n")
        #     print("【systemメッセージ】")
        #     print(system_content)
        #     print("\n【userメッセージ】")
        #     print(user_content)
        #     print("\n===== RAGによる回答生成 =====\n")
        #     print(answer)





100it [05:24,  3.24s/it]


In [22]:
prediction = query_df["answer"]
prediction.to_csv(csv_folder_path +"prediction.csv",header = False)